# Goal: 
### To identify all possible source of ghost rays during FOXSI2 observations Using FeXVIII Maps 
### Milo - Feb 2020



In [1]:
import matplotlib.animation as animation

In [ ]:
import glob
from sunpy.map import Map
from sunpy.time import TimeRange
from astropy.io import fits
from astropy.wcs import WCS
import astropy.units as u
from astropy.coordinates import SkyCoord
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import matplotlib.colors as colors
import matplotlib.dates as md
import matplotlib as mpl
mpl.rc('axes',labelsize=14)
mpl.rc('xtick',labelsize=12)
mpl.rc('ytick',labelsize=12)
from matplotlib.colors import ListedColormap
import numpy as np
import ipywidgets as widgets
%matplotlib inline
from ndcube import NDCube, NDCubeSequence
from sunpy.visualization.colormaps import cm
from scipy import ndimage
import sunpy

# FOXSI-2 List of Targets an Times
Ftarget = {'cen1_pos0':(359.0, -431.0), # arcsec
           'cen1_pos1':(-1.0, -431.0),
           'cen1_pos2':(-1.0, -251.0),
           'cen2_pos0':(-1.0, -281.0),
           'cen2_pos1':(-390.0, -281.0),
           'cen3_pos0':(1210.5, -431.5),
           'cen3_pos1':(850.0, -431.5),
           'cen3_pos2':(850.0, -251.0),
           'cen4':(200.0, 750.0),
           'cen5':(0.0, -251.0),
           'cen_Iris':(11,-260),
           'athiray':(30,-220)}
Ftimes = {'cen1_pos0': TimeRange(['2014-12-11 19:12:42', '2014-12-11 19:13:14.3']),
            'cen1_pos1':TimeRange(['2014-12-11 19:13:18','2014-12-11 19:13:42.6']),
            'cen1_pos2':TimeRange(['2014-12-11 19:13:46.5','2014-12-11 19:14:25']),
            'cen2_pos0':TimeRange(['2014-12-11 19:14:29','2014-12-11 19:14:39.6']),
            'cen2_pos1':TimeRange(['2014-12-11 19:14:44','2014-12-11 19:15:36.7']),
            'cen3_pos0':TimeRange(['2014-12-11 19:15:40.6','2014-12-11 19:16:07.2']),
            'cen3_pos1':TimeRange(['2014-12-11 19:16:11','2014-12-11 19:16:30.1']),
            'cen3_pos2':TimeRange(['2014-12-11 19:16:34','2014-12-11 19:17:09.2']),
            'cen4':TimeRange(['2014-12-11 19:17:13.5','2014-12-11 19:18:46.2']),
            'cen5':TimeRange(['2014-12-11 19:18:50.5','2014-12-11 19:19:23.2']),
            't_shtr':TimeRange(['2014-12-11 19:18:18','2014-12-11 19:18:22']),
            'Iris_Obs':TimeRange(['2014-12-11 19:12:22','2014-12-11 19:39:01'])}
fov = (1000,1000) # arcsec
# Reading AIA Maps:
data_dir = '/Volumes/Pandora/FOXSI/AIA/'
str_indices = {'94':'', '131':'', '171':'', '193':'', '211':'', '304':'', '335':'', '1600':'', '1700':''}
file_list = {}

for key in str_indices:
    file_list.update({key: glob.glob(data_dir + '*' + key + '*.fits')})
    file_list[key]=sorted(file_list[key])

F_AIA094 = file_list['94']
F_AIA171 = file_list['171']
F_AIA211 = file_list['211']
mapsFe18, mapsFe18_cores, labels, ns = [], [], [], []
i = 0
for f094, f171, f211 in zip(F_AIA094,F_AIA171,F_AIA211):
    i=i+1;print(i)
    m094  = Map(f094)
    m171  = Map(f171)
    m211  = Map(f211)
    mFe18 = Map(f171)
    mFe18.data[:] = m094.data - m211.data[:]/120 - m171.data[:]/450
    mask = mFe18.data < mFe18.max() * 0.08
    data = ndimage.gaussian_filter(mFe18.data * ~mask, 40)
    mFe18_core = sunpy.map.Map(data, mFe18.meta)
    label, n = ndimage.label(mFe18_core.data)
    mapsFe18.append(mFe18)
    mapsFe18_cores.append(mFe18_core)
    labels.append(label)

    ns.append(n)

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56


In [ ]:
fig, ax = plt.subplots(figsize=(10,10),subplot_kw=dict(projection=mapsFe18[0]));
l = 3
ims = []
for t in range (72-l,72+l):
    im = ax.imshow(mapsFe18[t].data)
    ax.set_title(t)
    #mapsFe18[t].plot_settings['norm'] = colors.Normalize()
    #.colorbar()
    #im.contour(labels[t])
    ims.append(im)

Ani = animation.ArtistAnimation(fig,ims,interval=5000,repeat_delay=1000)
#Ani.save('hola.mp4')
plt.show()

In [ ]:
def allfov(t):
    fig, ax = plt.subplots(figsize=(10,10),subplot_kw=dict(projection=mapsFe18[t]));
    mapsFe18[t].plot_settings['norm'] = colors.Normalize()
    mapsFe18[t].plot(vmin=0,vmax=100,cmap='viridis',title='FeXVIII    '+F_AIA171[t][41:63])
    plt.colorbar()
    plt.contour(labels[t])
    
def plotcores(t):
    fig, ax = plt.subplots(figsize=(10,10));
    mapsFe18_cores[t].plot_settings['norm'] = colors.Normalize()
    mapsFe18_cores[t].plot(vmin=0,vmax=10,cmap='viridis',title='FeXVIII    '+F_AIA171[t][41:63])
    plt.colorbar()



In [ ]:
l = 12; 
widgets.interact(plotcores, t=(68-l,72+l));
#widgets.interact(allfov, t=(68-l,72+l));

In [ ]:
# FeXVIII Map
tt = 79
fd094 = Map(file_list['94'][tt])
fd171 = Map(file_list['171'][tt])
fd211 = Map(file_list['211'][tt])
fd = Map(file_list['171'][tt])
fd.data[:] = fd094.data - fd211.data[:]/120 - fd171.data[:]/450

In [ ]:
F_AIA094 = file_list['94']
F_AIA171 = file_list['171']
F_AIA211 = file_list['211']

#F_AIA171 = file_list['171'][52:83]
#F_AIA211 = file_list['211'][56:87]
mapsFe18 = []

for f094, f171, f211 in zip(F_AIA094,F_AIA171,F_AIA211):
    m094  = Map(f094)
    m171  = Map(f171)
    m211  = Map(f211)
    mFe18 = Map(f171)
    mFe18.data[:] = m094.data - m211.data[:]/120 - m171.data[:]/450
    ## Co-rotating maps:
    top_right =   SkyCoord((Ftarget['athiray'][0] + athirayfov[0]/2) * u.arcsec, 
                           (Ftarget['athiray'][1] + athirayfov[1]/2) * u.arcsec, frame=mFe18.coordinate_frame)
    bottom_left = SkyCoord((Ftarget['athiray'][0] - athirayfov[0]/2) * u.arcsec, 
                           (Ftarget['athiray'][1] - athirayfov[1]/2) * u.arcsec,frame=mFe18.coordinate_frame)
    submap = mFe18.submap(bottom_left, top_right)
    mapsFe18.append(submap)    

In [ ]:
''' Plot '''
fig, ax = plt.subplots(figsize=(12,12),subplot_kw=dict(projection=fd))
fd.plot_settings['norm'] = colors.Normalize()
fd.plot(cmap='gnuplot2_r',vmin=0,vmax=100)
#fd.plot(cmap='gnuplot2_r',vmin=1,vmax=50)
plt.title('FeXVIII '+fd.name[-20:],fontsize=18)
plt.colorbar(label=fd.meta['pixlunit'])
plt.show()

In [ ]:
mask = fd.data < fd.max() * 0.08
data2 = ndimage.gaussian_filter(fd.data * ~mask, 40)
#data2[data2 < 100] = 0
fd2 = sunpy.map.Map(data2, fd.meta)
labels, n = ndimage.label(fd2.data)

In [ ]:
fig, ax = plt.subplots(figsize=(12,12),subplot_kw=dict(projection=fd))
fd.plot()
fd.plot_settings['norm'] = colors.Normalize()
fd.plot(cmap='gnuplot2_r',vmin=0,vmax=100)
plt.colorbar(label=fd.meta['pixlunit'])
plt.contour(labels)
plt.figtext(0.3, 0.2, f'Number of regions = {n}', color='white')
plt.title('AIA FeXVIII - '+fd.meta['t_obs'][:-4],fontsize='28')
ax.tick_params(labelsize=20)
plt.show()

In [ ]:
plt.figure(figsize=(10,10))
plt.imshow(data2,origin='lower')